In [1]:
import torch
import torchvision.datasets as dsets
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

import os
from PIL import Image
from torchvision import transforms

from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

In [2]:
import torch.utils.data as data
class ImageDataset(data.Dataset):
    def __init__(self, eq_path, an_path, json_path, lower_bound=0, upper_bound=1000, transform=None):
        self.eq_path = eq_path
        self.an_path = an_path
        self.json_path = json_path
        self.transform = transform
        self.eq_images = []
        self.an_images = []
        self.data_labels = []


        eq_filenames = os.listdir(eq_path)
        an_filenames = os.listdir(an_path)

        upper_bound = min(len(eq_filenames), len(an_filenames), upper_bound)

        for eq_filename in eq_filenames[lower_bound:upper_bound]:
            self.eq_images.append(eq_path + "/" + eq_filename)
            
        for an_filename in an_filenames[lower_bound:upper_bound]:
            self.an_images.append(an_path + "/" + an_filename)
        
        with open(csv_path) as f:
            df = pd.read_csv(f)
            self.data_labels = df["IsCorrectAnswer"]            

    def __len__(self):
        return len(self.eq_images)

    def __getitem__(self, index):
        eq_image = Image.open(self.eq_images[index])
        an_image = Image.open(self.an_images[index])
    
        if self.transform is None:
            self.transform = transforms.Compose([transforms.ToTensor()])

        eq_image = self.transform(eq_image).squeeze()
        an_image = self.transform(an_image).squeeze()
        image = torch.stack([eq_image, an_image])
        label = self.data_labels[index]
        return image, label

In [3]:
data = "/home/bakay/PycharmProjects/AI_project_final/pics/data"
labels = "/home/bakay/PycharmProjects/AI_project_final/pics/labels"
csv_path = "/home/bakay/PycharmProjects/AI_project_final/pics/info.csv"

In [4]:
class OlNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = nn.Conv2d(2, 32, 3, padding=2)
        self.conv_1_pool = nn.MaxPool2d(2)
        self.conv_2 = nn.Conv2d(32, 64, 3, padding=2)
        self.conv_2_pool = nn.MaxPool2d(2)
        self.conv_2_drop = nn.Dropout2d(0.25)
        self.conv_3 = nn.Conv2d(64, 128, 3, padding=2)
        self.conv_3_pool = nn.MaxPool2d(2)
        self.conv_4 = nn.Conv2d(128, 256, 3, padding=2)
        self.conv_4_pool = nn.MaxPool2d(2)
        self.conv_4_drop = nn.Dropout2d(0.25)
        self.fc_1 = nn.Linear(9216, 84)
        self.fc_2 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.conv_1_pool(F.relu(self.conv_1(x)))
        x = self.conv_2_drop(self.conv_2_pool(F.relu(self.conv_2(x))))
        x = self.conv_3_pool(F.relu(self.conv_3(x)))
        x = self.conv_4_drop(self.conv_4_pool(F.relu(self.conv_4(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc_1(x))
        x = F.softmax(self.fc_2(x))
        return x

In [5]:
lr = 0.0002
criterion = nn.CrossEntropyLoss()
model = OlNet()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [6]:
batch_size = 100
num_of_epochs = 100

In [7]:
train_dataset = ImageDataset(data, labels, csv_path, lower_bound=0, upper_bound=6000)
test_dataset = ImageDataset(data, labels, csv_path, lower_bound=6000, upper_bound=10000)
print(len(test_dataset))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# image, label = train_dataset[0]

4000


In [8]:
for epoch in range(num_of_epochs):
    # Training
    for i, (X, y_true) in enumerate(train_loader):

        minibatch = X.size(0)
        X = Variable(X)
        y_true = Variable(y_true)
        y_predict = model(X)
        print(y_true.size(), y_predict.size())
#         print(y_true.size())
        model.zero_grad()
        loss = criterion(y_predict, y_true)
        loss.backward()
        optimizer.step()
        print("Epoch: [{}/{}], Stage: [{}/{}], Loss: {}".format(epoch + 1, num_of_epochs, i + 1, len(train_loader), loss.data[0]))
#         break
    # Testing
    correct = np.array([])
    predicted = np.array([])
    for (X, y_true) in test_loader:
#         minibatch = X.size(0)
#         X = X.view(minibatch,)
        X = Variable(X)
        y_out = model(X)
        _, y_pred = torch.max(y_out, 1)
        correct = np.concatenate((correct, y_true.numpy()))
        predicted = np.concatenate((predicted, y_pred.data.numpy()))
    print(correct.shape, predicted.shape)
    print("Epoch: {}, Accuracy: {}%".format(epoch + 1, f1_score(correct, predicted)))

/home/bakay/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [1/60], Loss: 0.6927030086517334
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [2/60], Loss: 0.6933751702308655
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [3/60], Loss: 0.6879079937934875
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [4/60], Loss: 0.6988255381584167
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [5/60], Loss: 0.6885390281677246
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [6/60], Loss: 0.6948729157447815
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [7/60], Loss: 0.703656792640686
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [8/60], Loss: 0.7085170149803162
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [9/60], Loss: 0.6940041184425354
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/100], Stage: [10/60], Loss: 0.6933650374412537
torch.Size([100]) torch.Size([100, 2])
Epoch: [1/1

KeyboardInterrupt: 